In [1]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

In [2]:
# Define the input shape
input_shape = (150, 150, 3)

# Define the number of classes
num_classes = 4

In [3]:
# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define the paths to the training and test data
train_data_path = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_data_path = '/kaggle/input/brain-tumor-mri-dataset/Testing'

# Create the generators
train_generator = train_datagen.flow_from_directory(train_data_path, 
                                                    target_size=input_shape[:2], 
                                                    batch_size=32, 
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_data_path, 
                                                  target_size=input_shape[:2], 
                                                  batch_size=32, 
                                                  class_mode='categorical')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [4]:
# Define the ResNet model
resnet_model = tf.keras.applications.ResNet101(include_top=False, 
                                              weights='imagenet', 
                                              input_shape=input_shape)  

171446536/171446536 [==============================] - 1s 0us/step


In [5]:
# Add the classification layers on top of ResNet
classifier = keras.Sequential()
classifier.add(resnet_model)
classifier.add(layers.Flatten())
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dropout(0.5))
classifier.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Train the model
history = classifier.fit(train_generator, 
                         steps_per_epoch=train_generator.samples // train_generator.batch_size, 
                         epochs=50, 
                         validation_data=test_generator, 
                         validation_steps=test_generator.samples // test_generator.batch_size)

Epoch 1/50
178/178 [==============================] - 168s 516ms/step - loss: 1.6124 - accuracy: 0.6662 - val_loss: 3.5850 - val_accuracy: 0.2477
Epoch 2/50
178/178 [==============================] - 68s 381ms/step - loss: 0.9414 - accuracy: 0.7887 - val_loss: 5.7073 - val_accuracy: 0.2281
Epoch 3/50
178/178 [==============================] - 68s 381ms/step - loss: 0.5286 - accuracy: 0.8136 - val_loss: 1.4026 - val_accuracy: 0.2344
Epoch 4/50
178/178 [==============================] - 68s 379ms/step - loss: 0.5818 - accuracy: 0.8445 - val_loss: 3.1178 - val_accuracy: 0.3469
Epoch 5/50
178/178 [==============================] - 68s 378ms/step - loss: 0.4691 - accuracy: 0.8674 - val_loss: 1.2281 - val_accuracy: 0.4664
Epoch 6/50
178/178 [==============================] - 68s 380ms/step - loss: 0.4422 - accuracy: 0.8792 - val_loss: 0.6995 - val_accuracy: 0.7398
Epoch 7/50
178/178 [==============================] - 67s 376ms/step - loss: 0.2978 - accuracy: 0.9070 - val_loss: 0.4949 - val_a

In [7]:
classifier.save('/kaggle/working/Classifier.h5')

In [9]:
# Evaluate the model
score = classifier.evaluate(test_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Validation loss: 0.11681337654590607
Validation accuracy: 0.9649122953414917
